# Private 2nd Pytorch

원래 코드: https://dacon.io/competitions/official/235689/codeshare/2396?page=2&dtype=recent

ghghdfd 님의 tensorflow 코드를 pytorch로 재구현합니다.


- Pre-Learning 부분은 동일합니다.

    - 이에 대한 분석은 분석은 Review.pdf에서 확인하실 수 있습니다.

- Data Modeling 부분을 재구현하였습니다.


### Pre-Learning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

#### Data Loading

In [2]:
train=pd.read_csv('./open/train_features.csv')
train_labels=pd.read_csv('./open/train_labels.csv')
test=pd.read_csv('./open/test_features.csv')

submission=pd.read_csv('./open/sample_submission.csv')

pd.options.display.max_columns=50

#### Feature Engineering

In [3]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [4]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [5]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:33<00:00, 93.69it/s]


In [6]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|███████████████████████████████████████████████████████████████████████████████| 782/782 [00:06<00:00, 117.63it/s]


푸리에 변환

In [7]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [8]:
train=pd.concat(train_dt)

In [9]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|█████████████████████████████████████████████████████████████████████████████| 3125/3125 [00:12<00:00, 243.81it/s]


In [10]:
test=pd.concat(test_dt)

In [11]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|███████████████████████████████████████████████████████████████████████████████| 782/782 [00:01<00:00, 511.47it/s]


standard scailing

In [12]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [13]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col) 
# numpy 형태로 변형되어, 모델 돌리기 쉽게 하려고 DF로 변경

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)
# numpy 형태로 변형되어, 모델 돌리기 쉽게 하려고 다시 DF로 변경

In [14]:
train_sc

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,27.356382,8.807207,19.465910,0.376992,0.869226,0.150423,0.495681,-0.272719,-0.276391,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000101,0.001505,0.001501
1,0,1,-0.054866,0.833464,0.820412,-0.282128,-0.093560,0.011266,0.742974,-0.236152,-0.240632,0.416836,-0.118821,-0.255054,0.032738,-0.349095,0.377085,0.564992,0.166566,0.162871
2,0,2,0.024046,0.315921,0.081086,-0.182551,-0.053585,-0.003708,0.819822,-0.169815,-0.173080,0.086405,0.023750,-0.531727,-0.141582,-0.202368,-0.004887,0.175645,0.300944,0.306341
3,0,3,0.065632,0.117634,-0.040874,-0.194863,0.154242,0.005408,0.785669,-0.035229,-0.040560,-0.058780,-0.213920,0.285459,0.229520,-0.385106,-0.135647,-0.077915,0.609008,0.599518
4,0,4,0.151477,0.300751,0.317742,-0.350724,0.494539,0.154354,0.791528,0.021954,0.016872,0.039823,0.259227,-0.055206,0.057320,-0.174917,-0.028047,0.013483,0.259626,0.260669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,0.365037,0.011656,0.845701,0.080839,0.350395,0.112282,-0.138940,0.829394,0.823900,0.151679,0.037205,0.119409,-0.108728,-0.027804,-0.009085,-0.142794,0.063329,0.063674
1874996,3124,596,10.220817,5.476964,7.441373,3.605246,16.530576,11.843241,-0.167578,0.814816,0.809618,0.150658,-0.000363,0.265559,-0.027936,0.090560,-0.018412,-0.065316,-0.064300,-0.062949
1874997,3124,597,0.386337,0.177768,-0.080193,-0.192468,-0.033904,-0.227861,-0.151875,0.802027,0.797338,0.093524,-0.049283,0.260884,0.082744,0.123264,-0.152712,0.035970,-0.056225,-0.053918
1874998,3124,598,0.728823,0.014037,0.350745,0.136284,1.281790,0.403540,-0.175811,0.801880,0.797431,0.174681,-0.096564,0.071332,0.153722,-0.014412,-0.049662,-0.054574,0.000843,0.001922


## Modeling

In [30]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [31]:
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed

In [32]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [33]:
X=np.array(train_sc.iloc[:,2:]).reshape(-1, 18, 600)
X.shape
np.transpose(X, (1, 0, 2))
X.shape

(3125, 18, 600)

In [34]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(-1, 18, 600)
test_x.shape
np.transpose(test_x, (1, 0, 2))
test_x.shape

(782, 18, 600)

In [35]:
class cnn_model(torch.nn.Module):

    def __init__(self):
        
        super(cnn_model, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv1d(18, 128, kernel_size=9, padding = 4, stride=1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv1d(128, 256, kernel_size=7, padding = 3, stride=1),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.4))
            
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv1d(256, 128, kernel_size=3, padding = 1, stride=1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)

        self.avgpool = nn.AvgPool2d(8, stride=1)

        return out 
        # return self.log_softmax(x, dim=1)

In [36]:
model = cnn_model()

print(model)

cnn_model(
  (layer1): Sequential(
    (0): Conv1d(18, 128, kernel_size=(9,), stride=(1,), padding=(4,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv1d(128, 256, kernel_size=(7,), stride=(1,), padding=(3,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
  )
)


In [37]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the CPU


In [38]:
model = cnn_model().to(device)

In [39]:
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

In [40]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [54]:
y = train_labels['label'].values

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

y = to_categorical(train_labels['label'], 61)


y.shape

(3125, 61)

In [61]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)
reLR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5, patience = 4, verbose = True)
# reLR = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 4,verbose = 1,factor = 0.5) 

In [71]:
# from pytorchtools import EarlyStopping
# es = EarlyStopping(monitor='val_loss', patience=8, model='min')
# 왜인지 모르겠지만 pytorchtools에서 EarlyStopping을 import할 수 없으므로
# 직접 구현한 코드 들고 옴

In [72]:
class EarlyStopping:
    def __init__(self, patience=8, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [77]:
for epoch in range(0, training_epochs):
    
    print(f'Starting epoch {epoch + 1}')
    current_loss = 0.0

    for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
        
        inputs, targets = data
        optimzer.zero_grad()
        
        outputs = model(inputs)
        
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
            
    print('Training process has finished. Saving trained model.')
    print('Starting testing')
    
    save_path = f'./model-fold-{fold}.pth'
    torch.save(network.state_dict(), save_path)

    correct, total = 0, 0
    
    with torch.no_grad():

      for i, data in enumerate(testloader, 0):

        inputs, targets = data
        outputs = network(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)
        
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
        print(f'Average: {sum/len(results.items())} %')

Starting epoch 1


ValueError: too many values to unpack (expected 2)

In [74]:
es = EarlyStopping
# es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    mc = tf.keras.callbacks.ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model()
    
    
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    
    
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

--------------------Fold_1--------------------


AttributeError: 'cnn_model' object has no attribute 'fit'

기타 참고


- pytorch에서 EarlyStop 이용하기
    - https://quokkas.tistory.com/entry/pytorch%EC%97%90%EC%84%9C-EarlyStop-%EC%9D%B4%EC%9A%A9%ED%95%98%EA%B8%B0


- pytorch가 제공하는 Learning rate scheduler 정리 - ReduceLROnPlateau
    - https://sanghyu.tistory.com/113
    
    
- torch.optim.lr_scheduler.ReduceLROnPlateau 사용법
    - https://pytorch.org/docs/stable/optim.html
    
    
- 검증(validation) 추가하고 fit()와 get_data() 생성하기
    - https://deep-learning-study.tistory.com/351